<a href="https://colab.research.google.com/github/wairiukoirwine/ML_Mini_Model_Project/blob/main/Copy_of_mini_ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git config --global user.email "wairiukoirwine@gmail.com"
!git config --global user.name "wairiukoirwine"

In [ ]:
!git clone https://github.com/wairiukoirwine/ML_Mini_Model_Project.git
%cd ML_Mini_Model_Project

fatal: destination path 'ML_Mini_Model_Project' already exists and is not an empty directory.
/content/ML_Mini_Model_Project


In [ ]:
%%writefile house_prices.csv
Size_sqft,Bedrooms,Price
850,2,200000
900,2,210000
1000,3,250000
1200,3,280000
1500,4,350000
1700,4,400000
2000,5,500000
2200,5,550000
2500,6,600000
2700,6,650000


Overwriting house_prices.csv


In [2]:
import pandas as pd

# Create the dataset
data = {
    "Size_sqft": [850, 900, 1000, 1200, 1500, 1700, 2000, 2200, 2500, 2700],
    "Bedrooms": [2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
    "Price": [200000, 210000, 250000, 280000, 350000, 400000, 500000, 550000, 600000, 650000]
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv("house_prices.csv", index=False)

# Optional: if using Colab, download it
from google.colab import files
files.download("house_prices.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

MODEL_FILE = "house_price_model.joblib"

# Output widget
out = widgets.Output()

# Upload widgets
train_upload = widgets.FileUpload(accept=".csv", multiple=False, description="Upload CSV to Train")
predict_upload = widgets.FileUpload(accept=".csv", multiple=False, description="Upload CSV to Predict")

# Train model function
def train_model(df):
    X = df[["Size_sqft", "Bedrooms"]]
    y = df["Price"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    joblib.dump(model, MODEL_FILE)
    with out:
        clear_output()
        print("Model trained and saved as", MODEL_FILE)
        print("=== Model Evaluation ===")
        print("Mean Squared Error:", mse)
        print("R2 Score:", r2)
    return model

# Load model if exists
def load_model():
    if os.path.exists(MODEL_FILE):
        model = joblib.load(MODEL_FILE)
        with out:
            clear_output()
            print("Model loaded from", MODEL_FILE)
        return model
    else:
        with out:
            clear_output()
            print("No saved model found. Train a model first.")
        return None

# Predict function
def predict_from_csv(df, model):
    X_new = df[["Size_sqft", "Bedrooms"]]
    preds = model.predict(X_new)
    df["Predicted_Price"] = preds
    with out:
        clear_output()
        print("Predictions:")
        display(df)
    # Save predictions to CSV
    df.to_csv("predictions.csv", index=False)
    with out:
        print("Predictions saved as predictions.csv")

# Event handlers
def on_train_upload(change):
    if train_upload.value:
        uploaded_filename = list(train_upload.value.keys())[0]
        content = train_upload.value[uploaded_filename]["content"]
        df = pd.read_csv(io.BytesIO(content))
        train_model(df)

def on_predict_upload(change):
    if predict_upload.value:
        uploaded_filename = list(predict_upload.value.keys())[0]
        content = predict_upload.value[uploaded_filename]["content"]
        df = pd.read_csv(io.BytesIO(content))
        model = load_model()
        if model:
            predict_from_csv(df, model)

# Link widgets to events
train_upload.observe(on_train_upload, names="value")
predict_upload.observe(on_predict_upload, names="value")

# Display widgets
display(widgets.VBox([train_upload, predict_upload, out]))


In [ ]:
!pip install pandas scikit-learn joblib
!python ml_model.py


Do you want to train a new model? (yes/no)
yes
=== Model Evaluation ===
Mean Squared Error: 1300000000.0
R2 Score: 0.9658119658119658
Model saved as house_price_model.joblib
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
Input: [1800, 4] → Predicted Price: 400000.0
Input: [2500, 5] → Predicted Price: 550000.0
